In [1317]:
import pandas as pd
import numpy as np
import re

In [1318]:
df = pd.read_csv('../csv_files/properties.csv', low_memory=False)
df.head()

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
0,6,8901695,4180,4180.0,True,MIXED_USE_BUILDING,295000,NaN,3,242,...,False,True,36,True,1000,1403,1403,0,False,GOOD
1,6,8747010,8730,8730.0,True,VILLA,675000,NaN,4,349,...,False,False,0,True,977,1526,1526,0,False,AS_NEW
2,6,8775843,4020,4020.0,True,APARTMENT_BLOCK,250000,NaN,5,303,...,False,False,0,False,0,760,760,0,False,TO_RENOVATE
3,6,8910441,1200,1200.0,True,HOUSE,545000,NaN,4,235,...,False,False,0,False,0,63,63,0,False,JUST_RENOVATED
4,6,8758672,1190,1190.0,True,MIXED_USE_BUILDING,500000,NaN,2,220,...,False,False,0,True,60,193,193,0,False,AS_NEW


In [1319]:
df.shape

(75876, 22)

# Remove leading and trailing spaces from column names

In [1320]:
df.columns = [x.strip(' ') for x in df.columns.values]

# Remove leading and trailing spaces of every element

In [1321]:
# remove leading and trailing spaces and newline characters from values if they are a string
df = df.applymap(lambda x: x.strip() if type(x)==str else x)

#### 1. PostCode

In [1322]:
# 1 Converts postCode into int64

df['postcode'] = df['postcode'].astype('Int64')
df['postcode'].dtypes

Int64Dtype()

#### 2. Price

In [1323]:
# 2 Converting price
def grabs_strips(x):
    if type(x) == str:
        # return x.str.extract('(\d*\.?\d*)', expand=False).astype(float)
        return re.match(r'(\d*(,\d{3})*\.?\d*)', x).group()
    return x


df['price'] = df['price'].apply(grabs_strips)

In [1324]:
# Conversion into float
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [1325]:
df['price'].shape

(75876,)

In [1326]:
df.dropna(subset=['price'], inplace=True)
df.shape

(69778, 22)

## 3.house_is

In [1327]:
# fonction to Update the most relevant value of proprety_subtype

def updates_house_is(row):
    
    house_sub_type = ['HOUSE','house','VILLA','EXCEPTIONAL_PROPERTY', 'MANSION', 'villa', 'House', 'TOWN_HOUSE'
                      , 'Villa', 'COUNTRY-COTTAGE' ]
    
    app_sub_type = ['APARTEMENT', 'APARTEMENT','apartment','MIXED_USE_BUILDING','Apartment','DUPLEX','PENTHOUSE',
                   'APARTMENT_BLOCK','GROUND_FLOOR', 'duplex', 'ground-floor', 'Loft/Attic', 'APARTMENT_GROUP'
                    , 'Penthouse', 'penthouse', 'flat-studio', 'APARTMENT', 'apartement', 'Apartement']
    
    if row['property_subtype'] in house_sub_type:
        return True
    elif row['property_subtype'] in app_sub_type:
        return False
    return np.nan

# Storing in house_is prop of the df
df['house_is'] = df.apply(updates_house_is, axis=1)
df.house_is = df.house_is.astype('float64')

## 4.Sale

In [1328]:
df.sale = df['sale'].replace({
    'Wohnung': "Unknown",
    'Appartement': "Unknown",
    'Apartamento': "Unknown",
    '': "Unknown",
    'None': "Unknown",
    "unknown":"Unknown",
    "Maison":"Unknown",
    "Huis":"Unknown",
    "House":"Unknown"
})

df.sale = df['sale'].fillna('Unknown')

## 5.property_subtype

In [1329]:
df.property_subtype = df['property_subtype'].replace({
    'house': "HOUSE",
    'House': "HOUSE",
    'apartment': "APARTMENT",
    '': "Unknown",
    'villa': "VILLA",
    "duplex":"DUPLEX",
    "Huis":"HOUSE",
    "Maison":"HOUSE",
    'penthouse':'PENTHOUSE',
    'flat-studio':'FLAT_STUDIO',
    'ground-floor':'GROUND_FLOOR'
})

df.property_subtype = df['property_subtype'].fillna('Unknown')

## 9.room_number

In [1330]:
# replace None to np.nan
df.rooms_number.fillna(value=np.nan, inplace=True)
df.rooms_number[df.rooms_number==None]

Series([], Name: rooms_number, dtype: object)

In [1331]:
# replace 'None' to np.nan
df.rooms_number = df.rooms_number.apply(lambda x : np.nan if x=='None' else x)
df.rooms_number[df.rooms_number=='None']

Series([], Name: rooms_number, dtype: object)

In [1332]:
# change data type of rooms_number from object to float64
df.rooms_number = df.rooms_number.astype('float64')

## 10.area

In [1333]:
# remove 'm2' from value of area
df.area = df.area.replace("[^0-9.-]", "", regex=True)

In [1334]:
# replace 'm2' from value of area
df.area = df.area.replace('', np.nan)

In [1335]:
# replace None to np.nan
df.area.fillna(value=np.nan, inplace=True)

In [1336]:
# replace 'None' to np.nan
df.area = df.area.apply(lambda x : np.nan if x=='None' else x)

In [1337]:
# change data type from object to float64
df.area = df.area.astype('float64')

## 11.kitchen_has

In [1338]:
df.kitchen_has.value_counts(dropna=False)

True     49905
False    15562
NaN       4311
Name: kitchen_has, dtype: int64

In [1339]:
# change data type from object to float64
df.kitchen_has = df.kitchen_has.astype('float64')

In [1340]:
df.kitchen_has.value_counts(dropna=False)

1.0    49905
0.0    15562
NaN     4311
Name: kitchen_has, dtype: int64

## 12.furnished

In [1341]:
# change data type from object to float64
df.furnished = df.furnished.astype('float64')

## 13.open_fire

In [1342]:
# change data type from object to float64
df.open_fire = df.open_fire.astype('float64')

## 14.terrace

In [1343]:
# change numerical data to np.nan
df.terrace = df.terrace.replace(r'\d\.?\d?', True, regex=True)

In [1344]:
# replace string False to False
df.terrace = df.terrace.replace('False', False)

In [1345]:
# replace string False to False
df.terrace = df.terrace.replace('TRUE', True)

In [1346]:
# replace string False to False
df.terrace = df.terrace.replace('True', True)

In [1347]:
# change data type from object to bool
df.terrace = df.terrace.astype('float64')

In [1348]:
df.terrace.value_counts(dropna=False)

1.0    34219
0.0    25809
NaN     9750
Name: terrace, dtype: int64

## 15.terrace_area

In [1349]:
# replace 'None' to np.nan
df.terrace_area = df.terrace_area.apply(lambda x : np.nan if x=='None' else x)
df.terrace_area[df.terrace_area=='None']

Series([], Name: terrace_area, dtype: object)

In [1350]:
df.terrace_area = df.terrace_area.replace(True, np.nan)
df.terrace_area = df.terrace_area.replace('TRUE', np.nan)

In [1351]:
    # change data type from object to float64
    df.terrace_area = df.terrace_area.astype('float64')

## 16.Garden

In [1352]:
# replace string False to False
df.garden = df.garden.replace('False', False)

In [1353]:
# replace string False to False
df.garden = df.garden.replace('True', True)

In [1354]:
# change data type from object to bool
df.garden = df.garden.astype('float64')

## 17.Garden Area

In [1355]:
# replace None to np.nan
df.garden_area.fillna(value=np.NaN, inplace=True)
df.garden_area[df.garden_area==None]

# replace 'None' to np.nan
df.garden_area = df.garden_area.apply(lambda x : np.nan if x=='None' else x)
df.garden_area[df.garden_area=='None']

# change data type of rooms_number from object to float64
df.garden_area = df.garden_area.astype('float64')

## 18.land_surface

In [1356]:
# replace None to np.nan
df.land_surface.fillna(value=np.NaN, inplace=True)

# replace np.nan TO 0
df.land_surface = df.land_surface.replace(np.nan, 0)

# replace None to np.nan
df.land_surface.fillna(value=np.nan, inplace=True)

# replace 'None' to np.nan
df.land_surface = df.land_surface.apply(lambda x : np.nan if x=='None' else x)

# change data type of rooms_number from object to float64
df.land_surface = df.land_surface.astype('float64')

## 19.land_plot_surface

In [1357]:
# replace 'yes' from value to 0
df.land_plot_surface = df.land_plot_surface.replace("[^0-9.-]", "", regex=True)

# replace 'm2' from value of area
df.land_plot_surface = df.land_plot_surface.replace('', np.nan)

# replace None to np.nan
df.land_plot_surface.fillna(value=np.nan, inplace=True)

# replace 'None' to np.nan
df.land_plot_surface = df.land_plot_surface.apply(lambda x : np.nan if x=='None' else x)

# change data type of rooms_number from object to float64
df.land_plot_surface = df.land_plot_surface.astype('float64')

## 20.facades_number

In [1358]:
# replace 'None' to np.nan
df.facades_number = df.facades_number.apply(lambda x : np.nan if x=='None' else x)
df.facades_number[df.facades_number=='None']

# change data type of facades_number from object to float64
df.facades_number = df.facades_number.astype('float64')

## 21.swimming_pool_has

In [1359]:
 #change numerical data to np.nan
df.swimming_pool_has = df.swimming_pool_has.replace(r'\d\.?\d?', np.nan, regex=True)

# replace string False to False
df.swimming_pool_has = df.swimming_pool_has.replace('False', False)

# change data type from object to bool
df.swimming_pool_has = df.swimming_pool_has.astype('float64')

df.swimming_pool_has.value_counts(dropna=False)

0.0    59365
NaN     8451
1.0     1962
Name: swimming_pool_has, dtype: int64

## 22.building_state

In [1360]:
# change numerical data to np.nan
df.building_state = df.building_state.replace(r'\d\.?\d?', np.nan, regex=True)
df.building_state = df.building_state.apply(lambda x : np.nan if x=='None' else x)
df.building_state = df.building_state.replace(np.nan, 'Not specified')
df.building_state.unique()

array(['GOOD', 'AS_NEW', 'TO_RENOVATE', 'JUST_RENOVATED', 'TO_BE_DONE_UP',
       'TO_RESTORE', 'Not specified'], dtype=object)

# Check if there are columns with mixed data types ==> NO

In [1361]:
from pandas.api.types import infer_dtype
# print data type of each column to check if there are
# any mixed ones, turns out that there are none
def is_mixed(col):
    return infer_dtype(col)

df.apply(is_mixed)

# ==> there are no columns with 'mixed' part of the inferred datatype

source                integer
hyperlink              string
locality               string
postcode              integer
house_is             floating
property_subtype       string
price                floating
sale                   string
rooms_number         floating
area                 floating
kitchen_has          floating
furnished            floating
open_fire            floating
terrace              floating
terrace_area         floating
garden               floating
garden_area          floating
land_surface         floating
land_plot_surface    floating
facades_number       floating
swimming_pool_has    floating
building_state         string
dtype: object

## Find which kind of empties there are ==> there are only NaNs

In [1362]:
# are there any empty strings? ==> no
#print(np.where(df.applymap(lambda x: x == '')))

# are there any NaNs? ==> yes
np.where(pd.isnull(df))

(array([  366,   420,   550, ..., 69777, 69777, 69777]),
 array([ 4,  4,  4, ..., 18, 19, 20]))

# Display the percent of NaNs per column

In [1363]:
# display the percent of NaNs per column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'%_missing': percent_missing})
missing_value_df = missing_value_df.sort_values('%_missing', ascending = False)
missing_value_df

,%_missing
garden_area,32.752443
terrace_area,31.733498
land_plot_surface,27.958669
hyperlink,15.453295
facades_number,14.720972
land_surface,14.020178
terrace,13.972885
swimming_pool_has,12.111267
garden,10.622259
furnished,8.319241


### Put 'unknown' in place of NaN for everything else than int64 and float64 columns
### Please run this after converting numerical columns like price and facades from string to integer.
#### Even a value of NaN might help predict the price, so to avoid the correlation algorithm skipping it?, and because NaN is not allowed, we replace it.

In [1364]:
# replace all NaNs in strings with 'unknown'
df_nanfilled = df.select_dtypes(exclude=['int64','float64']).replace(np.nan, 'unknown')
df.update(df_nanfilled)

# replace all 'None'/'none' strings with uknown
df_nonefilled = df.select_dtypes(exclude=['int64','float64']).replace('none', 'unknown')
df.update(df_nonefilled)
df_nonefilled = df.select_dtypes(exclude=['int64','float64']).replace('None', 'unknown')
df.update(df_nonefilled)

In [1365]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69778 entries, 0 to 75875
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             69778 non-null  int64  
 1   hyperlink          69778 non-null  object 
 2   locality           69778 non-null  object 
 3   postcode           66731 non-null  Int64  
 4   house_is           65082 non-null  float64
 5   property_subtype   69778 non-null  object 
 6   price              69778 non-null  float64
 7   sale               69778 non-null  object 
 8   rooms_number       69563 non-null  float64
 9   area               66127 non-null  float64
 10  kitchen_has        65467 non-null  float64
 11  furnished          63973 non-null  float64
 12  open_fire          64257 non-null  float64
 13  terrace            60028 non-null  float64
 14  terrace_area       47635 non-null  float64
 15  garden             62366 non-null  float64
 16  garden_area        469

# Remove duplicates
### should execute after fixing columns
### should execute after removing non-property detail or incomplete columns: source and hyperlink

In [1366]:
# drop columns 
df.drop(['source', 'hyperlink'], axis = 1, inplace = True)

# drop 100% duplicate rows
lenght_before = len(df)
df.drop_duplicates(ignore_index = True, inplace = True)
dropped = len(df) - lenght_before
print(f'Dropped: {dropped}')

Dropped: -21652


## 6.locality and postcode
### Drop postcode column, because postcode is more completely available in 'locality'
### first we fix 'locality' column to carry just postcode or 'unknown' (stripping sporadic address parts)

In [1367]:
df.drop('postcode', axis = 1, inplace = True)

# write a function that returns the cleaned postcode from elements
# containing the address
def clean_locality(locality): 
    # Search for the presence of a 4 digit number (starts with 1-9)
    if re.search('[1-9]\d{3}', locality):
        # get the number
        return re.findall("[1-9]\d{3}", locality)[0]
    else: 
        # if no postcode is inside insert 'unknown' 
        return 'unknown'
          
# Updated locality column
df['locality'] = df['locality'].apply(clean_locality)

## 7.Create a region column

In [1368]:
def get_region(locality):
    if locality == 'unknown':
        return 'unknown'
    else:
        if not re.search('[1-9]\d{3}', locality):
            print('Please run this on already cleaned locality column')
            return 'unknown'
        elif int(locality) >= 1000 and int(locality) <=1299:
            return 'Brussels'
        elif int(locality) >= 1300 and int(locality) <=1499:
            return 'Wallonia'
        elif int(locality) >= 4000 and int(locality) <=7999:
            return 'Wallonia'
        else:
            return 'Flanders'
        
df['region'] = df['locality'].apply(get_region)

# Print unique values per column

In [1369]:
uniques = pd.DataFrame()
for col in df:
    col_uniques = pd.DataFrame({f'{col}_value': df[f'{col}'].value_counts().index,
                                f'{col}_count': df[f'{col}'].value_counts().values})
    uniques = pd.concat([uniques, col_uniques], axis = 1)

uniques.head(14)

,locality_value,locality_count,house_is_value,house_is_count,property_subtype_value,property_subtype_count,price_value,price_count,sale_value,sale_count,...,land_plot_surface_value,land_plot_surface_count,facades_number_value,facades_number_count,swimming_pool_has_value,swimming_pool_has_count,building_state_value,building_state_count,region_value,region_count
0,unknown,20197.0,1.0,24581.0,HOUSE,19177.0,295000.0,598.0,Unknown,37704.0,...,0.0,1902,0.0,17138.0,0.0,40604.0,Not specified,23831.0,unknown,20197.0
1,8300,1166.0,0.0,20721.0,APARTMENT,13752.0,199000.0,569.0,residential_sale,9537.0,...,100.0,353,2.0,10273.0,1.0,1466.0,AS_NEW,11075.0,Flanders,14462.0
2,1180,940.0,NaN,NaN,VILLA,3748.0,299000.0,562.0,first_session_with_reserve_price,622.0,...,90.0,313,4.0,7164.0,NaN,NaN,GOOD,7654.0,Wallonia,8438.0
3,1000,729.0,NaN,NaN,APARTMENT_BLOCK,1912.0,249000.0,558.0,Public Sale,172.0,...,120.0,291,3.0,5489.0,NaN,NaN,TO_BE_DONE_UP,2049.0,Brussels,5029.0
4,1050,684.0,NaN,NaN,MIXED_USE_BUILDING,1711.0,275000.0,555.0,annuity_monthly_amount,73.0,...,70.0,290,1.0,241.0,NaN,NaN,TO_RENOVATE,1777.0,NaN,NaN
5,9000,568.0,NaN,NaN,DUPLEX,1201.0,225000.0,538.0,Notary Sale,13.0,...,80.0,289,10.0,2.0,NaN,NaN,JUST_RENOVATED,1617.0,NaN,NaN
6,8400,454.0,NaN,NaN,PENTHOUSE,1078.0,395000.0,517.0,last_session_reached_price_min_overbid,2.0,...,110.0,263,6.0,1.0,NaN,NaN,TO_RESTORE,123.0,NaN,NaN
7,4000,330.0,NaN,NaN,GROUND_FLOOR,882.0,325000.0,457.0,final_public_sale,2.0,...,150.0,256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1150,297.0,NaN,NaN,EXCEPTIONAL_PROPERTY,678.0,250000.0,442.0,annuity_lump_sum,1.0,...,85.0,223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1200,291.0,NaN,NaN,FLAT_STUDIO,646.0,195000.0,442.0,NaN,NaN,...,75.0,210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1370]:
df.to_csv('../csv_files/cleaned_properties.csv')